In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gpflow
from gpflow.utilities import print_summary, positive
plt.style.use('ggplot')
%matplotlib inline
import tensorflow as tf
import math
from tensorflow import math as tm
from sklearn import preprocessing
from scipy.interpolate import interp1d

from tensorflow_probability import bijectors as tfb
from tensorflow_probability import distributions as tfd
from tensorflow_probability import mcmc
import tensorflow_probability as tfp

from reggae.data_loaders import load_barenco_puma
PI = tf.constant(math.pi, dtype='float64')
f64 = np.float64


In [ ]:
np.set_printoptions(threshold=np.inf)
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# k_exp.D[2] = 1.2849
# k_exp.D[3] = 1.0959
# k_exp.lengthscale.assign(1.4110)
# k_exp.h(2, 3).numpy()


# Replication of the paper by Lawrence et al., 2006

https://papers.nips.cc/paper/3119-modelling-transcriptional-regulation-using-gaussian-processes.pdf

#### Probesets

The original paper restricted their interest to 5 known targets of p53: 
- DDB2 -------------- (probeset 203409_at)
- p21 ----------------- (probeset 202284_s_at) (alias p21CIP1, CDKN1A)
- SESN1/hPA26 -- (probeset 218346_s_at)
- BIK ----------------- (probeset 205780_at)
- TNFRSF10b ----- (probeset 209294_x_at, 209295_at, 210405_x_at)

In [ ]:
m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()

m_df, m_observed = m_observed 
f_df, f_observed = f_observed
# Shape of m_observed = (replicates, genes, times)
m_observed = m_observed
f_observed = f_observed
σ2_m_pre = f64(σ2_m_pre)
σ2_f_pre = f64(σ2_f_pre)

display(m_df)



In [ ]:
# Copied from matlab
x = np.array([
[0.0969 , 0.0785 ,  0.0451 , 0.0001 , 0.0800],
[0.0998 , 0.0815 ,  0.0479 , 0.0014 , 0.0828],
[0.1088 , 0.0907 ,  0.0568 , 0.0054 , 0.0917],
[0.1245 , 0.1065 ,  0.0720 , 0.0123 , 0.1068],
[0.1470 , 0.1290 ,  0.0938 , 0.0220 , 0.1286],
[0.1766 , 0.1583 ,  0.1222 , 0.0346 , 0.1571],
[0.2133 , 0.1941 ,  0.1571 , 0.0498 , 0.1921],
[0.2568 , 0.2362 ,  0.1982 , 0.0676 , 0.2337],
[0.3071 , 0.2843 ,  0.2454 , 0.0879 , 0.2814],
[0.3638 , 0.3379 ,  0.2981 , 0.1103 , 0.3350],
[0.4265 , 0.3967 ,  0.3561 , 0.1348 , 0.3940],
[0.4950 , 0.4603 ,  0.4191 , 0.1611 , 0.4583],
[0.5691 , 0.5283 ,  0.4866 , 0.1890 , 0.5274],
[0.6486 , 0.6006 ,  0.5586 , 0.2186 , 0.6013],
[0.7335 , 0.6771 ,  0.6350 , 0.2497 , 0.6799],
[0.8239 , 0.7578 ,  0.7159 , 0.2825 , 0.7633],
[0.9199 , 0.8430 ,  0.8013 , 0.3169 , 0.8517],
[1.0218 , 0.9328 ,  0.8917 , 0.3530 , 0.9452],
[1.1300 , 1.0276 ,  0.9871 , 0.3910 , 1.0442],
[1.2445 , 1.1273 ,  1.0877 , 0.4310 , 1.1487],
[1.3655 , 1.2322 ,  1.1936 , 0.4728 , 1.2589],
[1.4928 , 1.3419 ,  1.3046 , 0.5165 , 1.3746],
[1.6260 , 1.4559 ,  1.4203 , 0.5618 , 1.4952],
[1.7642 , 1.5735 ,  1.5397 , 0.6083 , 1.6201],
[1.9064 , 1.6934 ,  1.6618 , 0.6555 , 1.7481],
[2.0508 , 1.8140 ,  1.7850 , 0.7028 , 1.8776],
[2.1956 , 1.9333 ,  1.9074 , 0.7492 , 2.0067],
[2.3385 , 2.0493 ,  2.0268 , 0.7939 , 2.1333],
[2.4770 , 2.1594 ,  2.1411 , 0.8358 , 2.2550],
[2.6086 , 2.2613 ,  2.2477 , 0.8740 , 2.3695],
[2.7308 , 2.3527 ,  2.3444 , 0.9075 , 2.4743],
[2.8412 , 2.4315 ,  2.4290 , 0.9355 , 2.5672],
[2.9378 , 2.4958 ,  2.4998 , 0.9572 , 2.6465],
[3.0189 , 2.5445 ,  2.5554 , 0.9722 , 2.7107],
[3.0836 , 2.5767 ,  2.5951 , 0.9802 , 2.7589],
[3.1312 , 2.5925 ,  2.6186 , 0.9814 , 2.7908],
[3.1620 , 2.5922 ,  2.6263 , 0.9759 , 2.8066],
[3.1766 , 2.5770 ,  2.6191 , 0.9644 , 2.8072],
[3.1762 , 2.5483 ,  2.5983 , 0.9475 , 2.7939],
[3.1625 , 2.5082 ,  2.5660 , 0.9262 , 2.7684],
[3.1377 , 2.4589 ,  2.5241 , 0.9014 , 2.7328],
[3.1039 , 2.4029 ,  2.4750 , 0.8744 , 2.6895],
[3.0635 , 2.3427 ,  2.4211 , 0.8460 , 2.6407],
[3.0190 , 2.2807 ,  2.3648 , 0.8175 , 2.5888],
[2.9727 , 2.2193 ,  2.3084 , 0.7898 , 2.5362],
[2.9268 , 2.1606 ,  2.2539 , 0.7637 , 2.4849],
[2.8833 , 2.1064 ,  2.2031 , 0.7400 , 2.4367],
[2.8438 , 2.0582 ,  2.1577 , 0.7193 , 2.3933],
[2.8099 , 2.0174 ,  2.1188 , 0.7021 , 2.3560],
[2.7827 , 1.9848 ,  2.0876 , 0.6888 , 2.3259],
[2.7632 , 1.9612 ,  2.0646 , 0.6795 , 2.3038],
[2.7519 , 1.9468 ,  2.0503 , 0.6743 , 2.2900],
[2.7491 , 1.9417 ,  2.0449 , 0.6732 , 2.2850],
[2.7549 , 1.9456 ,  2.0481 , 0.6760 , 2.2884],
[2.7689 , 1.9580 ,  2.0595 , 0.6824 , 2.3000],
[2.7904 , 1.9779 ,  2.0783 , 0.6919 , 2.3189],
[2.8185 , 2.0042 ,  2.1033 , 0.7040 , 2.3441],
[2.8517 , 2.0352 ,  2.1331 , 0.7180 , 2.3742],
[2.8884 , 2.0692 ,  2.1659 , 0.7330 , 2.4075],
[2.9265 , 2.1039 ,  2.1997 , 0.7481 , 2.4420],
[2.9637 , 2.1371 ,  2.2323 , 0.7623 , 2.4754],
[2.9977 , 2.1664 ,  2.2613 , 0.7747 , 2.5055],
[3.0258 , 2.1893 ,  2.2843 , 0.7840 , 2.5297],
[3.0457 , 2.2033 ,  2.2991 , 0.7894 , 2.5458],
[3.0551 , 2.2065 ,  2.3034 , 0.7900 , 2.5516],
[3.0519 , 2.1971 ,  2.2955 , 0.7851 , 2.5452],
[3.0345 , 2.1736 ,  2.2740 , 0.7742 , 2.5252],
[3.0020 , 2.1354 ,  2.2382 , 0.7570 , 2.4907],
[2.9537 , 2.0823 ,  2.1878 , 0.7336 , 2.4414],
[2.8900 , 2.0148 ,  2.1231 , 0.7041 , 2.3776],
[2.8117 , 1.9340 ,  2.0451 , 0.6693 , 2.3002],
[2.7200 , 1.8417 ,  1.9555 , 0.6299 , 2.2106],
[2.6171 , 1.7399 ,  1.8562 , 0.5868 , 2.1109],
[2.5053 , 1.6314 ,  1.7499 , 0.5414 , 2.0035],
[2.3874 , 1.5191 ,  1.6391 , 0.4947 , 1.8912],
[2.2665 , 1.4060 ,  1.5270 , 0.4482 , 1.7769],
[2.1455 , 1.2950 ,  1.4164 , 0.4031 , 1.6634],
[2.0274 , 1.1893 ,  1.3101 , 0.3607 , 1.5537],
[1.9150 , 1.0912 ,  1.2107 , 0.3220 , 1.4504],
[1.8107 , 1.0030 ,  1.1205 , 0.2879 , 1.3558],
[1.7166 , 0.9265 ,  1.0411 , 0.2591 , 1.2716],
[1.6342 , 0.8629 ,  0.9740 , 0.2360 , 1.1994],
[1.5645 , 0.8129 ,  0.9198 , 0.2189 , 1.1398],
[1.5079 , 0.7766 ,  0.8788 , 0.2077 , 1.0933],
[1.4645 , 0.7535 ,  0.8506 , 0.2021 , 1.0596],
[1.4337 , 0.7429 ,  0.8346 , 0.2017 , 1.0382],
[1.4145 , 0.7434 ,  0.8296 , 0.2058 , 1.0278],
[1.4056 , 0.7534 ,  0.8341 , 0.2138 , 1.0272],
[1.4054 , 0.7711 ,  0.8464 , 0.2247 , 1.0347],
[1.4121 , 0.7945 ,  0.8646 , 0.2378 , 1.0485],
[1.4238 , 0.8217 ,  0.8869 , 0.2520 , 1.0666],
[1.4387 , 0.8505 ,  0.9113 , 0.2667 , 1.0872],
[1.4548 , 0.8793 ,  0.9360 , 0.2809 , 1.1086],
[1.4705 , 0.9063 ,  0.9594 , 0.2940 , 1.1291],
[1.4844 , 0.9300 ,  0.9802 , 0.3054 , 1.1473],
[1.4951 , 0.9495 ,  0.9971 , 0.3146 , 1.1619],
[1.5016 , 0.9637 ,  1.0093 , 0.3214 , 1.1721],
[1.5032 , 0.9721 ,  1.0162 , 0.3254 , 1.1772],
[1.4994 , 0.9744 ,  1.0173 , 0.3267 , 1.1768],
[1.4900 , 0.9706 ,  1.0126 , 0.3252 , 1.1708]])

f = [0.0000, 0.0230,  0.0492,  0.0782,  0.1094,  0.1424,  0.1765,  0.2114,  0.2466,  0.2819,  0.3172,  0.3527,  0.3886,  0.4253,  0.4632,  0.5027,  0.5443,  0.5882,  0.6342,  0.6821,  0.7312,  0.7805,  0.8287,  0.8741,  0.9151,  0.9498,  0.9765,  0.9938,  1.0006,  0.9961,  0.9804,  0.9539,  0.9176,  0.8730,  0.8220,  0.7668,  0.7097,  0.6530,  0.5990,  0.5494,  0.5058,  0.4694,  0.4408,  0.4206,  0.4086,  0.4046,  0.4082,  0.4187,  0.4353,  0.4573,  0.4837,  0.5136,  0.5458,  0.5791,  0.6122,  0.6436,  0.6718,  0.6951,  0.7120,  0.7210,  0.7208,  0.7104,  0.6893,  0.6573,  0.6150,  0.5632,  0.5035,  0.4377,  0.3682,  0.2974,  0.2278,  0.1621,  0.1026,  0.0512,  0.0097, -0.0209, -0.0400, -0.0475, -0.0438, -0.0300, -0.0072,  0.0228,  0.0582,  0.0971,  0.1376,  0.1777,  0.2157,  0.2502,  0.2799,  0.3040,  0.3219,  0.3332,  0.3381,  0.3368,  0.3299,  0.3179,  0.3018,  0.2824,  0.2605,  0.2371]


b=[0.0020,0.0830,0.1508,0.2024,0.2373,0.2559,0.2591,0.2488,0.2283,0.2028,0.1804,0.1719,0.1851,0.2179,0.2612,0.3069,0.3493,0.3846,0.4102,0.4247,0.4277,0.4196,0.4021,0.3778,0.3510,0.3271,0.3123,0.3119,0.3270,0.3545,0.3887,0.4237,0.4548,0.4784,0.4920,0.4944,0.4852,0.4650,0.4355,0.3998,0.3622,0.3290,0.3077,0.3047,0.3214,0.3534,0.3935,0.4346,0.4716,0.5006,0.5191,0.5256,0.5197,0.5016,0.4729,0.4358,0.3943,0.3535,0.3207,0.3037,0.3075,0.3307,0.3668,0.4079,0.4472,0.4801,0.5033,0.5144,0.5125,0.4971,0.4689,0.4294,0.3811,0.3286,0.2787,0.2426,0.2334,0.2564,0.3031,0.3606,0.4194,0.4732,0.5179,0.5506,0.5693,0.5728,0.5606,0.5327,0.4899,0.4340,0.3683,0.2993,0.2407,0.2194,0.2578,0.3429,0.4521,0.5724,0.6971,0.8222]
fig = plt.figure(figsize=(14, 17))
for j in range(5):
    ax = plt.subplot(531+j)
    plt.plot(x[:,j], color='grey')
    plt.title(genes[0].index[j])
    plt.scatter(np.arange(0, 100, 100/7), Y[7*j:(j+1)*7], marker='x', label='Observed')
    plt.xticks([100/6 * i for i in range(7)])
    ax.axes.set_xticklabels(t)
    plt.xlabel('Time (h)')
plt.tight_layout()
    


# if plot_barenco:
#     barenco_f, _ = scaled_barenco_data(np.mean(f_samples[-10:], axis=0))
#     plt.scatter(τ[common_indices], barenco_f, marker='x', s=60, linewidth=3, label='Barenco et al.')
f = np.array(f)
b = np.array(b)
# plt.fill_between(τ, bounds[:, 0], bounds[:, 1], color='grey', alpha=0.3, label='95% credibility interval')
fig = plt.figure(figsize=(7, 5))
plt.xticks([100/6 * i for i in range(7)])
fig.axes[0].set_xticklabels(t)
plt.ylim((-0.4,1.4))
plt.xlabel('Time (h)')
plt.plot(f, c='cadetblue')
plt.fill_between(np.arange(0, 100), f+b, f-b, color='grey', alpha=0.3, label='2 x st. dev')

# print(genes[1])
print(np.arange(len(f))[::15].shape, tfs[1][0].shape)
tf = tfs[1][0]/np.mean(tfs[1][0]) * np.mean(f)
plt.scatter([100/6 * i for i in range(7)], tf, marker='x', s=70, label='Observed')
plt.legend()

In [ ]:
decay = np.array([0.331, 0.416, 0.392, 0.8, 0.441])[[0, 4, 2, 3, 1]]
sensi = np.array([0.825, 0.4846, 0.338, 1, 0.3292])[[0, 4, 2, 3, 1]]
basal = np.array([0.15, 0.05, 0.005, 0.005, 0.004])[[0, 4, 2, 3, 1]]
plt.figure(figsize=(7, 5))
plt.bar(np.arange(5)-0.3, basal, width=0.3, label='Basal rate')
plt.bar(np.arange(5), sensi, width=0.3, tick_label=genes[0].index, label='Sensitivity')
plt.bar(np.arange(5)+0.3, decay, width=0.3, color='blue', align='center', label='Decay rate')
plt.legend()

### Probeset Combination

TNFRSF10b has multiple probesets (probeset 209294_x_at, 209295_at, 210405_x_at) which should be combined.

It can be observed below that the log intensities have a similar pattern. Thus a popular way to combine is to take the average of the log intensities.

In [ ]:
COMBINE_MULTIPLE_PROBESETS = False
if COMBINE_MULTIPLE_PROBESETS:
    plt.figure(figsize=(10,7))
    plt.subplot(2,2,1)
    p53 = df[df.index.isin(['211300_s_at', '201746_at'])][columns].astype(float)
    for index, row in p53.iterrows():
        p53.loc[index] = np.log(list(row))
        plt.plot(list(row))

    p53_mean = pd.Series(p53.mean(0), index=genes.columns, name='p53')

    plt.subplot(2,2,2)
    TNFRSF10b = df[df.index.isin(['209294_x_at', '209295_at', '210405_x_at'])][columns]
    for index, row in TNFRSF10b.iterrows():
        print(list(row))
        TNFRSF10b.loc[index] = np.log(list(row))
        plt.plot(list(row))

    TNFRSF10b_mean = pd.Series(TNFRSF10b.mean(0), index=genes.columns, name='TNFRSF10b')
    
    plt.figure()
    plt.plot(p53_mean)

### Setup data

In [ ]:
num_genes = 5 #X.shape[1]
num_times = 7
replica = 0
print(m_observed.shape)
Y = m_observed[replica]
Y_var = σ2_m_pre[replica]
Y_orig_shape = Y.shape
Y = np.array([0,*Y.reshape(-1)]).reshape(-1, 1)
Y_var = Y_var.reshape(-1)
Sigma =tf.linalg.tensor_diag(tf.concat([[np.float64(0)], (Y_var.reshape(-1))], 0))
print(Sigma.dtype)
X = np.arange(num_times, dtype='float64')*2
X = np.c_[[X for _ in range(num_genes)]].reshape(-1, 1)
print(X[:8])
print(Y_var[:8])
Y = m_observed[0].reshape((-1, 1))

print(Y[:8])
print(X.shape,Y.shape, Y_var.shape)


### Model

We fix the sensitivity of p21 to be 1, and decay to be 0.8 as in Barenco et al.

            K_xx = np.zeros([X.shape[0],X.shape[0]], dtype='float64')
            print(K_xx.shape)
            for j in range(num_genes):
                for k in range(num_genes):
                    K_xx[j*block_size:(j+1)*block_size, 
                         k*block_size:(k+1)*block_size] = self.k_xx(j, k, X)
            return K_xx


In [ ]:
def broadcast_tile(a, h, w):
    x, y = a.shape
    m, n = x * h, y * w
    return tf.reshape(tf.broadcast_to(
        tf.reshape(a, (x, m//(h*x), y, n//(w*y))), (m//h, h, n//w, w)
    ), (m, n))

class ExpressionKernel(gpflow.kernels.Kernel):
    def __init__(self):
        super().__init__(active_dims=[0])
        
#         l_affine = tfb.AffineScalar(shift=tf.cast(1., tf.float64),
#                             scale=tf.cast(4-1., tf.float64))
#         l_sigmoid = tfb.Sigmoid()
#         l_logistic = tfb.Chain([l_affine, l_sigmoid])

        self.lengthscale = gpflow.Parameter(1.414, transform=positive())
#         self.white_variance = gpflow.Parameter(0.1, transform=positive())

        D_affine = tfb.AffineScalar(shift=tf.cast(0.1, tf.float64),
                                    scale=tf.cast(1.5-0.1, tf.float64))
        D_sigmoid = tfb.Sigmoid()
        D_logistic = tfb.Chain([D_affine, D_sigmoid])
        S_affine = tfb.AffineScalar(shift=tf.cast(0.1, tf.float64),
                                    scale=tf.cast(4.-0.1, tf.float64))
        S_sigmoid = tfb.Sigmoid()
        S_logistic = tfb.Chain([S_affine, S_sigmoid])

        self.D = gpflow.Parameter(np.random.uniform(0.9, 1, num_genes), transform=positive(), dtype=tf.float64)
#         self.D[3].trainable = False
#         self.D[3].assign(0.8)
        self.kervar = gpflow.Parameter(np.float64(1), transform=positive())
        self.S = gpflow.Parameter(np.random.uniform(1,1, num_genes), transform=positive(), dtype=tf.float64)
#         self.S[3].trainable = False
#         self.S[3].assign(1)
        self.noise_term = gpflow.Parameter(0.1353*tf.ones(num_genes, dtype='float64'), transform=positive())
        
    def K(self, X, X2=None):
        self.block_size = int(X.shape[0]/num_genes)
#         tf.print(self.D, self.S)
#         interp = interp1d(np.arange(Y_var.shape[0]), Y_var.reshape(-1), kind='linear')
#         Sigma = interp(np.linspace(0,Y_var.shape[0]-1, X.shape[0]))
#         Sigma = tf.linalg.tensor_diag(Sigma)
#         White = tf.linalg.diag(
#                     tf.fill(tf.shape(X)[:-1], tf.squeeze(self.white_variance)))

#         if X2 is None:
        shape = [X.shape[0],X.shape[0]]
        K_xx = self.k_xx(X, 0,0)        
        white = tf.linalg.diag(broadcast_tile(tf.reshape(self.noise_term, (1, -1)), 1, 7)[0])
        return K_xx + tf.linalg.diag((1e-5*tf.ones(X.shape[0], dtype='float64'))+Y_var) + white
#         else:
#             '''Calculate K_xf: no need to use tf.* since this part is not optimised'''
#             shape = [X.shape[0],X2.shape[0]]#self.block_size]

#             K_xf = tf.zeros(shape, dtype='float64')
#             for j in range(num_genes):
#                 mask = np.ones(shape)
#                 other = np.zeros(shape)
#                 mask[j*self.block_size:(j+1)*self.block_size] = 0
#                 pad_top = j*self.block_size
#                 pad_bottom = 0 if j == num_genes-1 else shape[0]-self.block_size-pad_top
#                 kxf = self.k_xf(j, X, X2)
#                 other = tf.pad(kxf,
#                                tf.constant([[pad_top,pad_bottom],[0,0]]), 'CONSTANT'
#                               )

#                 K_xf = K_xf * mask + other * (1 - mask)
#                 #[j*self.block_size:(j+1)*self.block_size] = 
#             return K_xf
        
    def k_xf(self, j, X, X2):
        t_prime, t_, t_dist = self.get_distance_matrix(t_x=X[:self.block_size].reshape(-1), 
                                                       t_y=X2)
        l = self.lengthscale
        erf_term = tm.erf(t_dist/l - self.gamma(j)) + tm.erf(t_/l + self.gamma(j))

        return self.S[j]*l*0.5*tm.sqrt(PI)*tm.exp(self.gamma(j)**2) *tm.exp(-self.D[j]*t_dist)*erf_term 

    def gamma(self):
        return self.D*self.lengthscale/2

    def h_(self, X, k, j, primefirst=True):
        l = self.lengthscale
#         print(l, self.D[k], self.D[j])
        t_x = X[:self.block_size].reshape(-1)
        t_prime, t, t_dist = self.get_distance_matrix(primefirst=primefirst, t_x=t_x)
        multiplier = tm.exp(self.gamma(k)**2) / (self.D[j]+self.D[k])
        first_erf_term = tm.erf(t_dist/l - self.gamma(k)) + tm.erf(t/l + self.gamma(k))
        second_erf_term = tm.erf(t_prime/l - self.gamma(k)) + tm.erf(self.gamma(k))
        return multiplier * (tf.multiply(tm.exp(-self.D[k]*t_dist) , first_erf_term) - \
                             tf.multiply(tm.exp(-self.D[k]*t_prime-self.D[j]*t) , second_erf_term))
    def h(self, X, k, j, primefirst=True):
        Dj = tf.reshape(self.D, (1, -1))
        Dj = broadcast_tile(Dj, 1, 7)
        Dj = tf.tile(Dj, [35, 1])
        Dk = tf.reshape(self.D, (-1, 1)) 
        Dk = broadcast_tile(Dk, 7, 1)
        Dk = tf.tile(Dk, [1, 35])
        gk = tf.transpose(broadcast_tile(tf.reshape(self.gamma(), (-1, 1)), 7, 1))
        gk = tf.tile(gk, [35, 1])
        if not primefirst:
            Dk, Dj = Dj, Dk
            gk = tf.transpose(broadcast_tile(tf.reshape(self.gamma(), (1,-1)), 1, 7))
            gk = tf.tile(gk, [1, 35])

        l = self.lengthscale
        t_x = tf.reshape(X[:self.block_size], (-1,))
        t_prime, t, t_dist = self.get_distance_matrix(primefirst=primefirst, t_x=t_x)
        t_prime = tf.tile(t_prime, [5, 5])
        t = tf.tile(t, [5, 5])
        t_dist = tf.tile(t_dist, [5, 5])
        multiplier = tm.exp(gk**2) / (Dj + Dk)
        first_erf_term = tm.erf(t_dist/l - gk) + tm.erf(t/l + gk)
        second_erf_term = tm.erf(t_prime/l - gk) + tm.erf(gk)
        return multiplier * (tf.multiply(tm.exp(-Dk*t_dist) , first_erf_term) - \
                             tf.multiply(tm.exp(-Dk*t_prime-Dj*t) , second_erf_term))


    def k_xx(self, X, j, k):
        S_square = tf.matmul(tf.reshape(self.S, (-1, 1)), tf.reshape(self.S, (1, -1)))
        S_square = broadcast_tile(S_square, 7, 7)
        mult = S_square*self.lengthscale*0.5*tm.sqrt(PI)
        return self.kervar**2*mult*(self.h(X, k, j) + self.h(X, j, k, primefirst=False))

    def get_distance_matrix(self, t_x, primefirst=True, t_y=None):
        if t_y is None:
            t_y = t_x
        t_1 = tf.transpose(tf.reshape(tf.tile(t_x, [t_y.shape[0]]), [ t_y.shape[0], t_x.shape[0]]))
        t_2 = tf.reshape(tf.tile(t_y, [t_x.shape[0]]), [ t_x.shape[0], t_y.shape[0]])
        if primefirst:
            return t_1, t_2, t_1-t_2
        return t_2, t_1, t_2-t_1
    
    def K_diag(self, X):
        print('k_diag')

        """I've used the fact that we call this method for K_ff when finding the covariance as a hack so
        I know if I should return K_ff or K_xx. In this case we're returning K_ff!!
        $K_{ff}^{post} = K_{ff} - K_{fx} K_{xx}^{-1} K_{xf}$"""
        _,_,t_dist = self.get_distance_matrix(t_x=X.reshape(-1))
        K_ff = tf.math.exp(-(t_dist**2)/(2*self.lengthscale**2))
        return (K_ff)


k_exp = ExpressionKernel()
print_summary(k_exp, fmt='notebook')
print(k_exp.K(X)[:7,:7])


In [ ]:
class MeanFunction(gpflow.mean_functions.MeanFunction):
    def __init__(self):
        affine = tfb.AffineScalar(shift=tf.cast(0., tf.float64),
                                  scale=tf.cast(2.-0., tf.float64))
        sigmoid = tfb.Sigmoid()
        logistic = tfb.Chain([affine, sigmoid])

        self.B = [gpflow.Parameter(np.mean(row)*k_exp.D[i], transform=positive()) for i, row in enumerate(Y.reshape(Y_orig_shape))]

    def __call__(self, X):
        ret = tf.zeros(0, dtype='double')
        block_size = int(X.shape[0]/num_genes)
        for j in range(num_genes):
            ret = tf.concat([ret, tf.repeat(self.B[j]/k_exp.D[j], block_size)], axis=0)
#         ret = tf.divide(self.B, (k_exp.D)).repeat(7)

        return ret[:,None]

meanfunc_exp = MeanFunction()
# meanfunc_exp(X)

In [ ]:
from gpflow.logdensities import multivariate_normal
class GPR(gpflow.models.GPR):
    def log_marginal_likelihood(self) -> tf.Tensor:
        r"""
        Computes the log marginal likelihood.

        .. math::
            \log p(Y | \theta).

        """
        X, Y = self.data
        K = self.kernel(X)
        m = self.mean_function(X)
        var = tf.linalg.diag(self.likelihood.variance*tf.ones(35, dtype='float64'))
        invK = tf.linalg.inv(K+var)
        L = tf.linalg.cholesky(K+var)
        log_prob = -0.5 * tf.linalg.matmul(tf.transpose(Y-m), tf.linalg.lstsq(tf.transpose(L), tf.linalg.lstsq(L, Y-m)))#tf.matmul(tf.matmul(tf.transpose(Y-m), invK), (Y-m))
        tf.print(log_prob)
        log_prob -= 0.5 * tf.reduce_sum(tm.log(tf.linalg.diag_part(L)))
        log_prob -= np.float64(35/2) * tm.log(2*PI)
#         num_data = Y.shape[0]
#         k_diag = tf.linalg.diag_part(K)
#         s_diag = tf.fill([num_data], self.likelihood.variance)
#         ks = tf.linalg.set_diag(K, k_diag + s_diag)
#         L = tf.linalg.cholesky(ks)
#         m = self.mean_function(X)
#         print(m)
        # [R,] log-likelihoods for each independent dimension of Y
        
#         log_prob = multivariate_normal(Y, m, L)
        return tf.reduce_sum(log_prob)

    def nll_stable(theta):
        # Numerically more stable implementation of Eq. (7) as described
        # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
        # 2.2, Algorithm 2.1.
        K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))
        L = cholesky(K)
        return np.sum(np.log(np.diagonal(L))) + \
               0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
               0.5 * len(X_train) * np.log(2*np.pi)

model = GPR(data=(X, Y), kernel=k_exp, mean_function=meanfunc_exp)

model.likelihood.variance.assign(3)
model.log_marginal_likelihood()

In [ ]:
opt = gpflow.optimizers.Scipy()
def objective_closure():
#     global prev_ret
#     try:
    ret = - model.log_marginal_likelihood()
#     except:


#     Ds.append([s.numpy() for s in model.kernel.D])

    tf.print(ret)
    return ret
opt_logs = opt.minimize(objective_closure,
                        model.trainable_variables,
                        options=dict(maxiter=14, disp=True, eps=0.00000001),
                        method='CG')

print_summary(model)


In [ ]:
model = gpflow.models.GPR(data=(X, Y), kernel=k_exp, mean_function=meanfunc_exp)

gpflow.config.set_default_jitter(np.float64(1e-3))

opt = gpflow.optimizers.Scipy()
print_summary(model)
Ds = list()

prev_ret = 0
def objective_closure():
#     global prev_ret
#     try:
    ret = - model.log_marginal_likelihood()
#     except:


#     Ds.append([s.numpy() for s in model.kernel.D])

    tf.print(ret)
    return ret


In [ ]:
opt_logs = opt.minimize(model.training_loss, model.trainable_variables, options=dict(maxiter=4, eps=1e-15))


In [ ]:
print_summary(model)


In [ ]:
        X, Y = self.data
        K = self.kernel(X)
        num_data = Y.shape[0]
        k_diag = tf.linalg.diag_part(K)
        s_diag = tf.fill([num_data], self.likelihood.variance)
        ks = tf.linalg.set_diag(K, k_diag + s_diag)
        L = tf.linalg.cholesky(ks)
        
        Linv = tf.linalg.triangular_solve(L, tf.eye((num_data), dtype='float64'), lower=True)
        Kinv = tf.transpose(Linv)*Linv 

#         m = self.mean_function(X)
# #         print(m)
#         # [R,] log-likelihoods for each independent dimension of Y
        
#         log_prob = multivariate_normal(Y, m, L)
        
        n = Y.shape[0];
        log_prob = -n*tf.math.log(2*PI) - tm.log(tf.reduce_prod(tf.linalg.diag_part(L))**2) - tf.transpose(Y)*Kinv*Y
        log_prob = log_prob*0.5;


In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.03)

def optimization_step(model):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        loss = - model.log_marginal_likelihood()
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


for epoch in range(80):
    l = optimization_step(model)
    if epoch % 10 == 0:
        print('Epoch ', epoch, ' loss', l)
        print_summary(model)


In [ ]:
print_summary(model)
print(model.kernel(X)[:7, :7])
# tf.linalg.cholesky(model.kernel(X)+100*tf.eye(35, dtype='float64'))

All the Hamiltonian MCMC sampling takes place in an unconstrained space (where constrained parameters have been mapped via a bijector to an unconstrained space). This makes the optimization, as required in the gradient step, much easier.

However, we often wish to sample the constrained parameter values, not the unconstrained one. The SamplingHelper helps us convert our unconstrained values to constrained parameter ones.

In general, adaptation prevents the chain from reaching a stationary distribution, so obtaining consistent samples requires num_adaptation_steps be set to a value somewhat smaller than the number of burnin steps

step_size: Larger step sizes lead to faster progress, but too-large step sizes make rejection exponentially more likely. When possible, it's often helpful to match per-variable step sizes to the standard deviations of the target distribution in each variable

num_leapfrog_steps: Integer number of steps to run the leapfrog integrator for. Total progress per HMC step is roughly proportional to step_size * num_leapfrog_steps.


In [ ]:
get_param_dists = True

if get_param_dists:
    #https://mc-stan.org/docs/2_22/stan-users-guide/fit-gp-section.html --> priors for length-scale
    model.kernel.lengthscale.prior = tfd.Uniform(f64(3.5), f64(144))
    model.likelihood.variance.prior = tfd.Gamma(f64(1.0), f64(1.0))
    for D in model.kernel.D:
        D.prior = tfd.Gamma(f64(1), f64(1))#tfd.Normal(f64(D.numpy()), f64(0.12))
    for S in model.kernel.S:
        S.prior = tfd.Gamma(f64(1), f64(1))#tfd.Normal(f64(S.numpy()), f64(0.12))
#     for B in model.mean_function.B:
#         B.prior = tfd.Gamma(f64(1), f64(1))#tfd.Normal(f64(B.numpy()), f64(0.12))
    print_summary(model)

    num_samples = 100
    num_burnin_steps = 10

    hmc_helper = gpflow.optimizers.SamplingHelper(
        target_log_prob_fn=model.log_marginal_likelihood, 
        model_parameters = model.trainable_parameters
    )

    hmc = mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=hmc_helper.target_log_prob_fn,
        num_leapfrog_steps=5,
        step_size=0.01
    )
    adaptive_hmc = mcmc.SimpleStepSizeAdaptation(
        hmc, #mcmc.TransformedTransitionKernel(, bijectors),
        num_adaptation_steps=int(num_burnin_steps*0.8),
        target_accept_prob=f64(0.75),
        adaptation_rate=0.1
    )
#     adaptive_hmc.bootstrap_results(hmc_helper.current_state)
    
# if False:
    @tf.function
    def run_chain_fn():
#         def trace_fn(something, pkr):
#             print(something)
#             print_summary(model)
#             return pkr.inner_results.is_accepted
        return mcmc.sample_chain(
            num_results=num_samples,
            num_burnin_steps=num_burnin_steps,
            current_state=hmc_helper.current_state,
            kernel=adaptive_hmc,
            trace_fn = lambda _, pkr: pkr.inner_results.is_accepted
        )

    samples, traces = run_chain_fn()
    parameter_samples = hmc_helper.convert_constrained_values(samples)


In [ ]:
param_to_name = {param: name for name, param in
                 gpflow.utilities.parameter_dict(model).items()}

D_mcmc = {f'.kernel.D[{i}]': 0 for i in range(num_genes)}
S_mcmc = {f'.kernel.S[{i}]':0 for i in range(num_genes)}
B_mcmc = {f'.mean_function.B[{i}]':0 for i in range(num_genes)}
lengthscale = 0

plt.figure(figsize=(8,4))

for val, param in zip(parameter_samples, model.parameters):
    name = param_to_name[param]
    plt.plot(tf.squeeze(val), label=name)

    if 'lengthscale' in name:
        lengthscale = np.std(val)
    elif 'kernel.D' in name:
        D_mcmc[name] = np.std(val)
    elif 'kernel.S' in name:
        S_mcmc[name] = np.std(val)
    elif 'function.B' in name:
        B_mcmc[name] = np.std(val)

plt.legend(bbox_to_anchor=(1., 1.))
plt.xlabel('hmc iteration')
plt.ylabel('parameter_values');

print(B_mcmc, list(S_mcmc.values()))
# plot_samples(samples, 'unconstrained_variables_values')

In [ ]:
# See page 130 Bayesian Stats for credible intervals (not implemented here yet)

plt.figure(figsize=(15, 10))
plt.subplot(3, 3, 1)
B = np.array([s.numpy() for s in model.mean_function.B])
S = np.array([s.numpy() for s in model.kernel.S])
D = np.array([d.numpy() for d in model.kernel.D])
B_barenco = np.array([2.6, 1.5, 0.5, 0.2, 1.35])
B_barenco = (B_barenco/np.mean(B_barenco)*np.mean(B))[[0, 4, 2, 3, 1]]
S_barenco = (np.array([3, 0.8, 0.7, 1.8, 0.7])/1.8)[[0, 4, 2, 3, 1]]
D_barenco = (np.array([1.2, 1.6, 1.75, 3.2, 2.3])*0.8/3.2)[[0, 4, 2, 3, 1]]


data = [B, S, D]
barenco_data = [B_barenco,  S_barenco, D_barenco]
vars = [0, 0,0]#[ S_mcmc, D_mcmc]
labels = ['Basal rates', 'Sensitivities', 'Decay rates']

plotnum = 331
for A, B, var, label in zip(data, barenco_data, vars, labels):
    plt.subplot(plotnum)
    plotnum+=1
    plt.bar(np.arange(5)-0.2, A, width=0.4, tick_label=genes[0].index)
    plt.bar(np.arange(5)+0.2, B, width=0.4, color='blue', align='center')

    plt.title(label)
#     plt.errorbar(np.arange(5)-0.2, A, yerr=list(var.values()), fmt='none', color='green', capsize=2)
# plt.bar(range(5), S, tick_label=genes.index[:num_genes])
# plt.title('')
# plt.errorbar(range(5), S, list(S_mcmc.values()), color='green')

# plt.subplot(3, 3, 3)
# plt.bar(range(5), D, tick_label=genes.index[:num_genes])
# plt.title()

In [ ]:
granularity = 100
pred_t = np.linspace(0, 12, granularity, dtype='float64')
print(int(X.shape[0]/num_genes))
k = model.kernel
Kxx = k.K(X, None)
K_inv = tf.linalg.inv(Kxx)
Kxf = k.K(X, pred_t)
print(Kxf.shape, K_inv.shape)
# x = tf.convert_to_tensor(Y) - np.repeat(B/np.array(D), 7).reshape(-1, 1)
KfxKxx = tf.matmul(tf.transpose(Kxf), K_inv)
mu_post = tf.matmul(KfxKxx, Y) #replace with x?

print(tf.transpose(Kxf).shape, '*', K_inv.shape,'==', KfxKxx.shape)
print('mu shape', mu_post.shape)

mu_post = mu_post.numpy()
mu_post -= mu_post[0]
plt.plot(mu_post)

scale_pred = np.sqrt(np.var(mu_post));
barencof = np.array([[0.0, 200.52011, 355.5216125, 205.7574913, 135.0911372, 145.1080997, 130.7046969],
                     [0.0, 184.0994134, 308.47592, 232.1775328, 153.6595161, 85.7272235, 168.0910562],
                     [0.0, 230.2262511, 337.5994811, 276.941654, 164.5044287, 127.8653452, 173.6112139]])


barencof = barencof[0]/(np.sqrt(np.var(barencof[0])))*scale_pred;
lb = len(barencof)
plt.scatter(np.arange(lb) * granularity/lb, barencof, marker='x')



In [ ]:
num_data = X.shape[0]
k_diag = tf.linalg.diag_part(Kxx)
s_diag = tf.fill([num_data], model.likelihood.variance)
ks = tf.linalg.set_diag(Kxx, k_diag + s_diag)
L = tf.linalg.cholesky(ks)
mu = model.mean_function(X)
#         print(m)
        # [R,] log-likelihoods for each independent dimension of Y
d = Y - mu
alpha = tf.linalg.triangular_solve(L, d, lower=True)
plt.figure(figsize=(13, 13))

for j in range(5):
    plt.subplot(531+j)
    plt.scatter(np.arange(7), Y[7*j:(j+1)*7])
    plt.plot(alpha[7*j:(j+1)*7])

In [ ]:
scale_pred = np.sqrt(np.var(mu_post));
barencof = np.array([[0.0, 200.52011, 355.5216125, 205.7574913, 135.0911372, 145.1080997, 130.7046969],
                     [0.0, 184.0994134, 308.47592, 232.1775328, 153.6595161, 85.7272235, 168.0910562],
                     [0.0, 230.2262511, 337.5994811, 276.941654, 164.5044287, 127.8653452, 173.6112139]])


barencof = barencof[0]/(np.sqrt(np.var(barencof[0])))*scale_pred;
# measured_p53 = df[df.index.isin(['211300_s_at', '201746_at'])][columns].astype(float)
# measured_p53 = measured_p53.mean(0)
# measured_p53 = measured_p53*scale_pred

lb = len(barencof)
# plt.scatter(np.arange(lb) * 100/lb, barencof, marker='x')
# for j in range(num_genes):
#     plt.figure()
#     plt.plot(mu_post[j*7:(j+1)*7])
#     plt.scatter(range(7), Y[j*7:(j+1)*7])


# plt.ylim(-1, 4);

# Kff = k.K_diag(X)
# K_post = Kff - KfxKxx * Kxf
# print('K shape', K_post.shape)

In [ ]:
mean, var = model.predict_f(X)
mean.shape

In [ ]:
plt.subplot(2, 2, 1)
plt.plot(mean)
plt.subplot(2, 2, 2)
plt.plot(genes[genes.index == 'p53'].values[0])

## Non-linear response in GRN Inference notebook
    

Investigate using https://gpflow.readthedocs.io/en/master/notebooks/advanced/varying_noise.html